In [38]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Download and load the data into a pandas dataframe
1. I've copied the data and created a csv file out of it. I'm fetching it from a URL hosted on the dropbox public folder.

In [39]:
!wget -q -O 'toronto_data.csv' https://www.dropbox.com/s/yxodhl9t7z8s2z5/toronto.csv?dl=0
dfToronto = pd.read_csv('toronto_data.csv')

# Data preparation
1. Remove the 'Not assigned' Borough entries from the dataframe as their geolocation would not be looked up.
2. Update Neighbourhood with Borough value where Neighbourhood is 'Not assigned'.
3. Grouped the neighbourhoods, against Postcode and Borough.

In [40]:
dfToronto = dfToronto.drop(dfToronto[dfToronto.Borough == 'Not assigned'].index)
dfToronto.loc[dfToronto['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = dfToronto['Borough']
dfGrouped = dfToronto.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [41]:
dfGrouped.shape

(103, 3)

# Download and load the geo coordinates data

In [42]:
#!wget -q -O 'toronto_geo_data.csv' https://www.dropbox.com/s/rd9rtsnx3nz9bfh/Geospatial_Coordinates.csv?dl=0
dfGeoData = pd.read_csv('toronto_geo_data.csv')
dfGeoData.set_index('Postal Code', inplace=True)

In [43]:
geolocator = Nominatim(user_agent="toronto_explorer")
aLatitude = []
aLongitude = []

for postcode, borough in zip(dfGrouped['Postcode'], dfGrouped['Borough']):
    aLatitude.append(dfGeoData.loc[postcode,:]['Latitude'])
    aLongitude.append(dfGeoData.loc[postcode,:]['Longitude'])

dfGrouped['Latitude'] = aLatitude
dfGrouped['Longitude'] = aLongitude
    
dfGrouped.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [44]:
dfGrouped.shape

(103, 5)